In [1]:
import re

import pdfplumber
import pandas as pd
from collections import namedtuple

In [2]:
Line = namedtuple('Line', 'company_id company_name doctype reference currency voucher inv_date due_date open_amt_tc open_amt_bc current months1 months2 months3')

In [4]:
company_re = re.compile(r'(V\d+) (.*) Phone:')
line_re = re.compile(r'\d{2}/\d{2}/\d{4} \d{2}/\d{2}/\d{4}')

In [14]:
file = 'Sample Report Pythonic.pdf'

In [15]:
lines = []
total_check = 0

with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()
        for line in text.split('\n'):
            comp = company_re.search(line)
            if comp:
                vend_no, vend_name = comp.group(1), comp.group(2)

            elif line.startswith('INVOICES'):
                doctype = 'INVOICE'

            elif line.startswith('CREDITNOTES'):
                doctype = 'CREDITNOTE'

            elif line_re.search(line):
                items = line.split()
                lines.append(Line(vend_no, vend_name, doctype, *items))
                
            elif line.startswith('Supplier total'):
                tot = float(line.split()[2].replace(',', ''))
                total_check += tot

In [18]:
df = pd.DataFrame(lines)
df.head()

,company_id,company_name,doctype,reference,currency,voucher,inv_date,due_date,open_amt_tc,open_amt_bc,current,months1,months2,months3
0,V50012,Company A LTD,CREDITNOTE,478,USD,BFS030OB/000000057,11/03/2014,12/30/2014,263.52,263.52,0.00,0.00,0.00,263.52
1,V50012,CARGO SAFETRAVEL LTD,INVOICE,J1062628/1,USD,BFS026/000001137,03/30/2019,03/30/2019,-102.00,-102.00,0.00,-102.00,0.00,0.00
2,V50012,CARGO SAFETRAVEL LTD,INVOICE,J1062638/1,USD,BFS026/000001136,02/28/2019,03/15/2019,-204.00,-204.00,0.00,-204.00,0.00,0.00
3,V50014,ChaseB REGISTER USDA,INVOICE,110-135633,USD,BFS026/000001064,02/05/2019,04/29/2019,"-4,665.00","-4,665.00","-4,665.00",0.00,0.00,0.00
4,V50015,METALS CHEMICALS GROUP LIMITED,INVOICE,SI191433,USD,BFS026/000000908,02/12/2019,03/30/2019,-198.00,-198.00,0.00,-198.00,0.00,0.00


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 14 columns):
company_id      131 non-null object
company_name    131 non-null object
doctype         131 non-null object
reference       131 non-null object
currency        131 non-null object
voucher         131 non-null object
inv_date        131 non-null datetime64[ns]
due_date        131 non-null datetime64[ns]
open_amt_tc     131 non-null float64
open_amt_bc     131 non-null float64
current         131 non-null float64
months1         131 non-null float64
months2         131 non-null float64
months3         131 non-null float64
dtypes: datetime64[ns](2), float64(6), object(6)
memory usage: 14.5+ KB


In [20]:
df['inv_date'] = pd.to_datetime(df['inv_date'])
df['due_date'] = pd.to_datetime(df['due_date'])

In [22]:
for col in df.columns[-6:]:
    df[col] = df[col].map(lambda x: float(str(x).replace(',', '')))

In [24]:
df['open_amt_bc'].sum()

-129580.73999999999

In [25]:
total_check

-129580.73999999999

In [26]:
df.to_csv('invoices.csv', index=False)